Lambda School Data Science

*Unit 2, Sprint 3, Module 3*

---

In [ ]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*
    !pip install eli5

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [ ]:
!pip install category_encoders==2.*
!pip install eli5

# Boosting and Permutation Importance

- Use xgboost for **gradient boosting**
- Get **permutation importances** for model interpretation and feature selection

In [ ]:
from category_encoders import OrdinalEncoder
from sklearn.ensemble import GradientBoostingClassifier # New model
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance # For interpretability
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier # Another implementation of new model
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['CommViolPredUnnormalizedData.txt']), na_values=['?'])

## Wrangle Function

In [ ]:
# def wrangle(fm_path, tv_path=None):
#     if tv_path:
#         df = pd.merge(pd.read_csv(fm_path, parse_dates=['date_recorded'], na_values=[-2e-08, 0]), 
#                          pd.read_csv(tv_path)).set_index('date_recorded')

#     else:
#         df = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv', parse_dates=['date_recorded']).set_index('date_recorded')
    
#     # When columns have zeros and shouldn't, they are like null values.
#     # So we will replace the zeros with nulls, and impute missing values later.
#     # Also create a "missing indicator" column, because the fact that
#     # values are missing may be a predictive signal.
#     cols_with_zeros = ['longitude', 'latitude', 'construction_year', 
#                        'gps_height', 'population']
#     for col in cols_with_zeros:
#         df[col] = df[col].replace(0, np.nan)
#         df[col+'_MISSING'] = df[col].isnull()
            
#     # Drop duplicate columns
#     duplicates = ['quantity_group', 'payment_type']
#     df = df.drop(columns=duplicates)
    
#     # Drop recorded_by (never varies) and id (always varies, random)
#     unusable_variance = ['recorded_by', 'id']
#     df = df.drop(columns=unusable_variance)
    
    
#     # Edftract components from date_recorded, then drop the original column
#     df['year_recorded'] = df.index.year
#     df['month_recorded'] = df.index.month
#     df['day_recorded'] = df.index.day

    # # Engineer feature: how many years from construction_year to date_recorded
    # df['years'] = df['year_recorded'] - df['construction_year']
    # df['years_MISSING'] = df['years'].isnull()
    
    # # return the wrangled dataframe
    # return df

In [ ]:
df.head()

,BerkeleyHeightstownship,NJ,39,5320,1,11980,3.1,1.37,91.78,6.5,1.88,12.47,21.44,10.93,11.33,11980.1,100,75122,89.24,1.55,70.2,23.62,1.03,18.39,79584,29711,30233,13600,5725,27101,5115,22838,227,1.96,5.81,9.9,48.18,2.7,64.55,14.65,...,?.4,?.5,?.6,?.7,?.8,?.9,?.10,?.11,?.12,?.13,?.14,?.15,?.16,6.5.1,1845.9,9.63,?.17,?.18,?.19,?.20,0.2,?.21,0.3,0.4,0.5,0.6,1.1,8.2,4,32.81,14.1,114.85,138,1132.08,16,131.26,2,16.41,41.02,1394.59
0,Marpletownship,PA,45.0,47616.0,1,23123,2.82,0.80,95.57,3.44,0.85,11.01,21.30,10.48,17.18,23123,100.0,47917,78.99,1.11,64.11,35.50,2.75,22.85,55323,20148,20191,18137,0,20074,5250.0,12222,885,3.98,5.61,13.72,29.89,2.43,61.96,12.26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.6,2186.7,3.84,NaN,NaN,NaN,NaN,0.0,NaN,0,0.00,1.0,4.25,5.0,21.26,24.0,102.05,57.0,242.37,376.0,1598.78,26.0,110.55,1.0,4.25,127.56,1955.95
1,Tigardcity,OR,NaN,NaN,1,29344,2.43,0.74,94.33,3.43,2.35,11.36,25.88,11.01,10.28,29344,100.0,35669,82.00,1.15,55.73,22.25,2.94,14.56,42112,16946,17103,16644,21606,15528,5954.0,8405,1389,4.75,2.80,9.09,30.13,4.01,69.80,15.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.6,2780.9,4.37,NaN,NaN,NaN,NaN,0.0,NaN,3,8.30,6.0,16.60,56.0,154.95,14.0,38.74,274.0,758.14,1797.0,4972.19,136.0,376.30,22.0,60.87,218.59,6167.51
2,Gloversvillecity,NY,35.0,29443.0,1,16656,2.40,1.70,97.35,0.50,0.70,12.55,25.20,12.19,17.57,0,0.0,20580,68.15,0.24,38.95,39.48,11.71,18.33,26501,10810,10909,9984,4941,3541,2451.0,4391,2831,17.23,11.05,33.68,10.81,9.86,54.74,31.22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.2,3217.7,3.31,NaN,NaN,NaN,NaN,0.0,NaN,0,0.00,10.0,57.86,10.0,57.86,33.0,190.93,225.0,1301.78,716.0,4142.56,47.0,271.93,NaN,NaN,306.64,NaN
3,Bemidjicity,MN,7.0,5068.0,1,11245,2.76,0.53,89.16,1.17,0.52,24.46,40.53,28.69,12.65,0,0.0,17390,69.33,0.55,42.82,32.16,11.21,14.43,24018,8483,9009,887,4425,3352,3000.0,1328,2855,29.99,12.15,23.06,25.28,9.08,52.44,6.89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.5,974.2,0.38,NaN,NaN,NaN,NaN,0.0,NaN,0,0.00,NaN,NaN,4.0,32.04,14.0,112.14,91.0,728.93,1060.0,8490.87,91.0,728.93,5.0,40.05,NaN,9988.79
4,Springfieldcity,MO,NaN,NaN,1,140494,2.45,2.51,95.65,0.90,0.95,18.09,32.89,20.04,13.26,140494,100.0,21577,75.78,1.00,41.15,29.31,7.12,14.09,27705,11878,12029,7382,10264,10753,7192.0,8104,23223,17.78,8.76,23.03,20.66,5.72,59.02,14.31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.4,1995.7,0.97,NaN,NaN,NaN,NaN,0.0,NaN,7,4.63,77.0,50.98,136.0,90.05,449.0,297.29,2094.0,1386.46,7690.0,5091.64,454.0,300.60,134.0,88.72,442.95,6867.42


In [ ]:
df.shape

(2214, 147)

In [ ]:
df.columns=['communityname','State','countyCode','communityCode','fold','pop','perHoush','pctBlack','pctWhite','pctAsian','pctHisp','pct12-21',
            'pct12-29','pct16-24','pct65up','persUrban','pctUrban','medIncome','pctWwage','pctWfarm','pctWdiv','pctWsocsec','pctPubAsst','pctRetire'
            ,'medFamIncome','perCapInc','whitePerCap','blackPerCap','NAperCap','asianPerCap','otherPerCap','hispPerCap','persPoverty','pctPoverty'
            ,'pctLowEdu','pctNotHSgrad','pctCollGrad','pctUnemploy','pctEmploy','pctEmployMfg','pctEmployProfServ','pctOccupManu','pctOccupMgmt'
            ,'pctMaleDivorc','pctMaleNevMar','pctFemDivorc','pctAllDivorc','persPerFam','pct2Par','pctKids2Par','pctKids-4w2Par','pct12-17w2Par'
            ,'pctWorkMom-6','pctWorkMom-18','kidsBornNevrMarr','pctKidsBornNevrMarr','numForeignBorn','pctFgnImmig-3','pctFgnImmig-5','pctFgnImmig-8'
            ,'pctFgnImmig-10','pctImmig-3','pctImmig-5','pctImmig-8','pctImmig-10','pctSpeakOnlyEng','pctNotSpeakEng','pctLargHousFam','pctLargHous'
            ,'persPerOccupHous','persPerOwnOccup','persPerRenterOccup','pctPersOwnOccup','pctPopDenseHous','pctSmallHousUnits','medNumBedrm','houseVacant',
            'pctHousOccup','pctHousOwnerOccup','pctVacantBoarded','pctVacant6up','medYrHousBuilt','pctHousWOphone','pctHousWOplumb','ownHousLowQ'
            ,'ownHousMed','ownHousUperQ','ownHousQrange','rentLowQ','rentMed','rentUpperQ','rentQrange','medGrossRent','medRentpctHousInc','medOwnCostpct'
            ,'medOwnCostPctWO','persEmergShelt','persHomeless','pctForeignBorn','pctBornStateResid','pctSameHouse-5','pctSameCounty-5','pctSameState-5'
            ,'numPolice','policePerPop','policeField','policeFieldPerPop','policeCalls','policCallPerPop','policCallPerOffic','policePerPop2'
            ,'racialMatch','pctPolicWhite','pctPolicBlack','pctPolicHisp','pctPolicAsian','pctPolicMinority','officDrugUnits','numDiffDrugsSeiz'
            ,'policAveOT','landArea','popDensity','pctUsePubTrans','policCarsAvail','policOperBudget','pctPolicPatrol','gangUnit','pctOfficDrugUnit'
            ,'policBudgetPerPop','murders','murdPerPop','rapes','rapesPerPop','robberies','robbbPerPop','assaults','assaultPerPop','burglaries'
            ,'burglPerPop','larcenies','larcPerPop','autoTheft','autoTheftPerPop','arsons','arsonsPerPop','violentPerPop','nonViolPerPop']


In [ ]:
df = df.dropna(axis=0, how="any")

In [ ]:
df.head()

,communityname,State,countyCode,communityCode,fold,pop,perHoush,pctBlack,pctWhite,pctAsian,pctHisp,pct12-21,pct12-29,pct16-24,pct65up,persUrban,pctUrban,medIncome,pctWwage,pctWfarm,pctWdiv,pctWsocsec,pctPubAsst,pctRetire,medFamIncome,perCapInc,whitePerCap,blackPerCap,NAperCap,asianPerCap,otherPerCap,hispPerCap,persPoverty,pctPoverty,pctLowEdu,pctNotHSgrad,pctCollGrad,pctUnemploy,pctEmploy,pctEmployMfg,...,policeCalls,policCallPerPop,policCallPerOffic,policePerPop2,racialMatch,pctPolicWhite,pctPolicBlack,pctPolicHisp,pctPolicAsian,pctPolicMinority,officDrugUnits,numDiffDrugsSeiz,policAveOT,landArea,popDensity,pctUsePubTrans,policCarsAvail,policOperBudget,pctPolicPatrol,gangUnit,pctOfficDrugUnit,policBudgetPerPop,murders,murdPerPop,rapes,rapesPerPop,robberies,robbbPerPop,assaults,assaultPerPop,burglaries,burglPerPop,larcenies,larcPerPop,autoTheft,autoTheftPerPop,arsons,arsonsPerPop,violentPerPop,nonViolPerPop
53,Toledocity,OH,95.0,77000.0,1,332943,2.54,19.70,76.96,1.05,3.97,14.78,28.65,14.52,12.25,332943,100.0,24819,71.86,0.36,36.37,28.93,12.29,18.44,30980,11894,13077,7722,9450,12415,7272.0,7817,62426,19.09,8.57,26.84,14.06,9.92,55.56,19.58,...,351211.0,108407.80,514.2,210.8,97.91,78.77,16.98,4.10,0.0,21.08,30.0,12.0,89.3,83.5,3988.5,3.00,222.0,41348280.0,80.67,5.0,4.39,127629.2,45,13.89,357.0,110.19,1594.0,492.02,1195.0,368.86,5502.0,1698.29,15251.0,4707.51,4517.0,1394.26,357.0,110.19,984.96,7910.25
54,Philadelphiacity,PA,101.0,60000.0,1,1585577,2.63,39.86,53.52,2.74,5.63,13.92,28.02,14.12,13.74,1585577,100.0,24603,70.12,0.35,32.15,31.63,13.98,18.20,30140,12091,15027,9061,10146,8285,5083.0,6053,313374,20.27,11.29,35.69,15.22,9.62,52.77,13.58,...,5480855.0,358261.41,840.2,426.4,79.54,73.48,23.18,2.84,0.0,25.95,273.0,14.0,39.9,140.0,11326.0,29.31,822.0,287578496.0,99.94,0.0,4.19,187978.5,439,28.70,785.0,51.31,11531.0,753.74,6821.0,445.86,15117.0,988.14,39181.0,2561.10,23785.0,1554.73,2282.0,149.17,1279.60,5253.14
57,Lynchburgcity,VA,680.0,47672.0,1,66049,2.63,26.41,72.45,0.76,0.72,17.79,29.84,18.19,14.33,66049,100.0,23726,72.25,0.63,39.06,35.02,7.54,17.20,30141,12657,14721,7200,12833,6139,8400.0,7231,9889,16.45,13.61,30.52,21.74,5.97,56.27,19.31,...,61164.0,91072.10,443.2,205.5,81.87,90.58,9.42,0.00,0.0,9.42,8.0,8.0,38.7,51.2,1290.5,3.33,62.0,5585480.0,89.13,0.0,5.80,83166.8,3,4.47,45.0,67.00,107.0,159.32,351.0,522.63,560.0,833.83,2316.0,3448.48,184.0,273.97,50.0,74.45,753.42,4630.73
98,NewHaventown,CT,9.0,52070.0,1,130474,2.66,36.14,53.85,2.41,13.22,17.06,34.55,19.17,10.81,130474,100.0,25811,73.04,0.47,37.07,26.28,14.61,12.75,31163,12968,16127,9616,7750,11739,7014.0,7420,25481,21.29,11.86,28.97,26.70,9.25,56.71,16.41,...,145000.0,121233.40,371.8,326.1,74.62,79.23,17.44,3.33,0.0,20.77,12.0,10.0,167.4,19.5,6679.9,9.54,100.0,17939552.0,89.74,5.0,3.08,149991.2,22,18.39,130.0,108.69,1238.0,1035.08,1154.0,964.85,3417.0,2856.93,7719.0,6453.80,1873.0,1566.00,134.0,112.04,2127.02,10988.76
121,Springfieldcity,OH,23.0,74118.0,1,70487,2.59,17.38,81.61,0.51,0.65,16.05,28.26,15.37,13.59,70487,100.0,21407,69.94,0.31,30.37,31.35,14.47,20.99,26838,10648,11196,8112,5418,15847,3530.0,7654,13999,20.87,9.79,31.67,10.88,8.74,53.25,23.26,...,108785.0,153872.80,1077.1,142.9,86.22,94.06,3.96,1.98,0.0,5.94,6.0,9.0,52.8,20.2,3485.1,1.06,45.0,5653511.0,93.07,0.0,5.94,79967.1,6,8.49,59.0,83.45,208.0,294.21,786.0,1111.77,963.0,1362.13,4048.0,5725.76,457.0,646.41,6.0,8.49,1497.92,7742.79


In [ ]:
df.shape

(111, 147)

# Split Data

In [ ]:
target = 'violentPerPop'
X = df.drop(columns=target)
y = df[target]

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Establish Baseline

In [ ]:
print('Baseline Accuracy Score:', y_train.value_counts(normalize=True).max())

Baseline Accuracy Score: 0.011363636363636364


# Build Model

In [ ]:
# model_skgb = make_pipeline(
#     OrdinalEncoder(),
#     SimpleImputer(),
#     GradientBoostingClassifier(random_state=42)
# )

# model_skgb.fit(X_train, y_train);

ValueError: ignored

In [ ]:
model_xgb = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    XGBClassifier(random_state=42,
                  n_jobs=-1)
)

model_xgb.fit(X_train, y_train);

# Check Metrics

In [ ]:
print('sklearn Training Accuracy:', model_skgb.score(X_train, y_train))
print('sklearn Validation Accuracy:', model_skgb.score(X_val, y_val))

In [ ]:
print('XGBoost Training Accuracy:', model_xgb.score(X_train, y_train))
print('XGBoost Validation Accuracy:', model_xgb.score(X_val, y_val))

ValueError: ignored

# Tuning / Communication

- How can we determine or communicate which features are most important to our model when making predictions?

**Option 1:** Grab feature importances from our pipeline

- **Plus:** Less computationally expensive.
- **Minus:** Can be misleading.


In [ ]:
importances = model_xgb.named_steps['xgbclassifier'].feature_importances_
feature_names = X_train.columns
feat_imp =  pd.Series(importances, index=feature_names).sort_values()

feat_imp.tail(10).plot(kind='barh')
plt.xlabel('Gini Importance')
plt.ylabel('Feature')

**Option 2:** Drop-column Importance

- Good, but computationally expensive because you have to train a model for each feature.

In [ ]:
col = 'quantity'

model_w_col = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    XGBClassifier(n_estimators=25,
                  random_state=42,
                  n_jobs=-1)
)

model_w_col.fit(X_train, y_train);

print(f'Training Accuracy w/ "{col}" included:', model_w_col.score(X_val, y_val))


In [ ]:
model_wo_col = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    XGBClassifier(n_estimators=25,
                  random_state=42,
                  n_jobs=-1)
)

model_wo_col.fit(X_train.drop(columns=col), y_train);

print(f'Training Accuracy w/ "{col}" excluded:', 
      model_wo_col.score(X_val.drop(columns=col), y_val))

In [ ]:
print(f'Drop-column importance of "{col}":', 0.72037-0.678872)

In [ ]:
X_train.shape

**Option 3:** Permutation Importance

In [ ]:
# By hand

# Step 1: Choose my feature
col = 'quantity'

# Step 2: Train model w/ ALL features
model_xgb = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    XGBClassifier(n_estimators=25,
                  random_state=42,
                  n_jobs=-1)
)

model_xgb.fit(X_train, y_train);

# Step 3: Evaluate model using VALIDATION DATA.
print('XGBoost Validation Accuracy:', model_xgb.score(X_val, y_val))

# Step 4: In VALIDATION DATA, permute the feature we're evaluating
X_val_perm = X_val.copy()
X_val_perm[col] = np.random.permutation(X_val_perm[col])

# Step 5: Calculate our error metric with the permuted data 
print(f'XGBoost Validation Accuracy w "{col}" permuted:', 
      model_xgb.score(X_val_perm, y_val))

In [ ]:
# Using sklearn

perm_imp = permutation_importance(
    model_xgb,
    X_val, 
    y_val,
    n_repeats=5,
    n_jobs=-1,
    random_state=42
)

In [ ]:
data = {'imp_mean': perm_imp['importances_mean'],
      'imp_std': perm_imp['importances_std']}

df = pd.DataFrame(data, index=X_val.columns).sort_values('imp_mean')

In [ ]:
df

In [ ]:
df['imp_mean'].tail(10).plot(kind='barh')